In [1]:
import mdtraj as md
import pyemma as pm

from pathlib import Path
import os
import pickle
import pandas 
import numpy as np

from openmm import app
import openmm as mm
import openmm.unit as unit
import openmmtools

import openpathsampling as ops
import openpathsampling.engines.openmm as ops_openmm
from openpathsampling.engines.openmm.tools import ops_load_trajectory
from openpathsampling.engines.openmm import trajectory_from_mdtraj

from cv import *

In [12]:
# Paths
title = 'CLN_dbscan(0.06,50)_tps'
model_path = Path('./data/CLN_msm/')
md_path = Path('./data/init_paths/CLN_md/')
storage_path = Path(f'./data/{title}')
storage_path.mkdir(parents=True, exist_ok=True)

# Parameters
f_scheme = 'ca'
n_mcsteps = 10

In [3]:
# Read tica_mod, kmeans_mod, msm, and clustering

with open(os.path.join(model_path, 'msm_models'), 'rb') as f:
    models = pickle.load(f)
tica_mod = models['tica_mod']
kmeans_mod = models['kmeans_mod']
msm = models['msm']

with open(os.path.join(model_path, 'clustering'), 'rb') as f:
    models = pickle.load(f)
dbscan_mod = models['dbscan']

In [4]:
# Prepare CVs
# Define the intersection of volumes using tics

tica_1 = ops.CoordinateFunctionCV("tica_1", tica_cv,
                                  tica_mod=tica_mod,
                                  f_scheme=f_scheme,
                                  element=0)

tica_2 = ops.CoordinateFunctionCV("tica_2", tica_cv,
                                  tica_mod=tica_mod,
                                  f_scheme=f_scheme,
                                  element=1)

dbscan_pre = ops.CoordinateFunctionCV('dbscan_pre', dbscan_predict,
                                      tica_1=tica_1,
                                      tica_2=tica_2,
                                      dbscan_mod=dbscan_mod)

# Define metastable state hypervolumes
unfolded = ops.volume.CVDefinedVolume(collectivevariable=dbscan_pre, lambda_min=-0.1, lambda_max=0.1).named('unfolded')
folded = ops.volume.CVDefinedVolume(collectivevariable=dbscan_pre, lambda_min=0.9, lambda_max=1.1).named('folded')

In [5]:
gro = app.GromacsGroFile(os.path.join(md_path, 'nvt.gro'))
# Gmx topology only stores reference to ff definitions. Need to specifty the directory contains ff files.
top = app.GromacsTopFile(os.path.join(md_path, 'topol.top'), 
                         periodicBoxVectors=gro.getPeriodicBoxVectors(),
                         includeDir=r'/home/rzhu/Loc/gromacs/share/gromacs/top')

# Create system from gmx files
system = top.createSystem(nonbondedMethod=app.PME,
                          nonbondedCutoff=1.0*unit.nanometers,
                          constraints=app.HBonds, 
                          rigidWater=True,
                          ewaldErrorTolerance=0.0005)

# Velocity-Verlet with Velocity Randomisation Integrator that simulates Langvein dynamics
integrator = openmmtools.integrators.VVVRIntegrator(
    # Temperature
    temperature=340*unit.kelvin,
    # Collision rate (temperature coupling time constant)
    collision_rate=1.0/unit.picoseconds,
    # Timestep
    timestep=2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

In [6]:
# Parallelisation may slow down simulation. It's dependent on the system size (the larger the better), connection speed between gpus (nv>>pcie), gpu power,
# and specific forces. 

openmm_properties = {'DeviceIndex':'2'}
platform = mm.openmm.Platform.getPlatformByName('CUDA')
engine_options = {
    # Allowed returned traj frames
    'n_frames_max': 1000,
    # Time interval between frames = 1 ps
    'n_steps_per_frame': 500
}
ops_topology = ops_openmm.tools.topology_from_pdb(os.path.join(md_path, 'nvt.gro'))
engine = ops_openmm.Engine(
    topology=ops_topology, 
    system=system, 
    integrator=integrator, 
    openmm_properties=openmm_properties,
    options=engine_options
).named('CLN_Openmm_engine')
engine.initialize(platform)

In [7]:
# Define the path ensemble to be sampled from
network = ops.TPSNetwork(initial_states=[folded, unfolded], 
                         final_states=[folded, unfolded])
# Define shooting scheme. A shooting move scheme can contains multiple shooting strategies 
scheme = ops.OneWayShootingMoveScheme(network=network, 
                                      selector=ops.UniformSelector(),
                                      engine=engine).named("CLN_bidir_scheme")

In [8]:
# Load initial trajectory from external files
init_traj=trajectory_from_mdtraj(md.load(os.path.join(md_path, 'trans.pdb')))
initial_conditions = scheme.initial_conditions_from_trajectories(init_traj)

No missing ensembles.
No extra ensembles.


In [30]:
# This is to start a new storage
# storage = ops.Storage(storage_path.joinpath(f'{title}.nc'), "w", template=init_traj[1])
sampler = ops.PathSampling(storage=storage,
                           move_scheme=scheme,
                           sample_set=initial_conditions)

In [ ]:
# This cell is to restart a simulation 

storage = ops.Storage(storage_path.joinpath(f'{title}.nc'), "a", template=init_traj[1])
sampler.run(5000)
storage.close()

Working on Monte Carlo cycle number 7
Starting simulation...
Working on first step


In [ ]:
sampler.restart_at_step(storage.steps[-1])